# Writing all_data CSV

In [ ]:
import pandas as pd
from core import Config
# Read data from disk
config = Config()
historic_frame: pd.DataFrame = pd.read_csv(config.filtered_dir / 'filtered_historic.csv', index_col=[0, 1])
static_dtypes: pd.DataFrame = pd.read_csv(config.filtered_dir / 'static_dtypes.csv')
static_frame: pd.DataFrame = pd.read_csv(config.filtered_dir / 'manually_filtered_static.csv', dtype=static_dtypes.values, index_col=0)
# seperate dataframes by instrument
histordict: dict[str, pd.DataFrame] = {}
statdict: dict[str, pd.DataFrame] = {}
for instrument, df in historic_frame.groupby(level=0):
    histordict.update({instrument: df})
for instrument, df in static_frame.groupby(level=0):
    statdict.update({instrument: df})
# Join historic and static dataframes
#all_dataframes: dict[str, pd.DataFrame] = {}
#for instrument, historic_df in histordict.items():
#    all_dataframes[instrument] = historic_df.join(statdict[instrument])
#all_data: pd.DataFrame = pd.concat(all_dataframes.values())
#all_data.to_csv(config.dataset_dir / 'all_data.csv')
del historic_frame, static_frame, static_dtypes

# Loading All Data

In [ ]:
import pandas as pd
from core import Config
config = Config()
all_data: pd.DataFrame = pd.read_csv(config.dataset_dir / 'all_data.csv', index_col=[0, 1])

# Split Training and Validation Set

In [ ]:
y: pd.DataFrame = all_data['TR.UpstreamScope3PurchasedGoodsAndServices'].to_frame()
X: pd.DataFrame = all_data.drop('TR.UpstreamScope3PurchasedGoodsAndServices', axis=1)

In [ ]:
cat_features: list[int] = list(range(0, X.shape[1]))

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=100, nan_mode='Min')
model.fit(X, y, cat_features=cat_features, verbose=True)